<a href="https://colab.research.google.com/github/taddbackus/capstone/blob/main/Working%20RNN/Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [ ]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 13.6 MB/s eta 0:00:00


In [ ]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [ ]:
pip install eemont

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135742 sha256=184f630b13105a9957499bcb01f8a4ca439eceff2ed04aa540c881cee9a8db57
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236755 sha256=255c103eeb94a75c910f86825e0b9f8a5adeb228123cc66f79a2866f12879f66
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [ ]:
import eeconvert as eec

In [ ]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#join in overall info
# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/OSMRE_GEOMINE_GEOJSON.geojson')
mines = pd.DataFrame(mines)
mines = mines[['company',
               'coalmine_op_status',
               'mine_name',
               'permit_id',
               'state_tribe_mine_id',
               'national_id',
               'coal_bed_names',
               'inspectable_unit_status',
                'post_smcra',
               'calculated_area',
               'reported_area',
               'permit_application_type',
               'permit_application_date',
               'permit_approval_date',
               'edit_date',
               'area_mine',
               'contour',
               'mountaintop',
               'steep_slope',
               'highwall',
               'auger',
               'contact',
              'information_link']]

In [ ]:
mines.columns

In [ ]:
mines.head()

In [ ]:
import pandas as pd
import os

directory_path = '/content/drive/MyDrive/Mines_Data/'
directory_files = os.listdir(directory_path)

df = pd.DataFrame()
z=0
for i in directory_files:
    df_file = pd.read_csv(os.path.join(directory_path, i), encoding = 'ISO-8859-1',low_memory=False)
    df_file['year'] = directory_files[z]
    df_file['year'] = df_file['year'].str.slice(3, 7)
    df_file['end_date'] = df_file['year']+'-07-31'
    df_file['start_date'] = df_file['year']+'-07-01'
    df_file['start_date'] = pd.to_datetime(df_file['start_date'])
    df_file['end_date'] = pd.to_datetime(df_file['end_date'])

    df_file['State'] = directory_files[z]
    df_file['State'] = df_file['State'].str.slice(0, 2)

    df = pd.concat([df, df_file])

    z+=1


In [ ]:
#filtering down to mines present in all years for RNN
all_year_mines = df['mine_id'][df['year']=='2017'].unique()
df = df.query('mine_id in @all_year_mines')
all_year_mines = df['mine_id'][df['year']=='2019'].unique()
df = df.query('mine_id in @all_year_mines')
all_year_mines = df['mine_id'][df['year']=='2021'].unique()
df = df.query('mine_id in @all_year_mines')
all_year_mines = df['mine_id'][df['year']=='2023'].unique()
df = df.query('mine_id in @all_year_mines')

In [ ]:
#mapping demo in python https://colab.research.google.com/github/QuantEcon/quantecon-notebooks-datascience/blob/master/applications/maps.ipynb#scrollTo=SI5i7rboSGFL&uniqifier=2
# Read the downloaded file

0	#419bdf	water
1	#397d49	trees
2	#88b053	grass
3	#7a87c6	flooded_vegetation
4	#e49635	crops
5	#dfc35a	shrub_and_scrub
6	#c4281b	built
7	#a59b8f	bare
8	#b39fe1	snow_and_ice

label0 = df[['orig_perm_id', 'Company','mine_id','water', 'start_date']].groupby(['orig_perm_id','mine_id', 'start_date','Company']).mean().reset_index()
label1 = df[['orig_perm_id', 'Company','mine_id', 'trees', 'start_date']].groupby(['orig_perm_id','mine_id','start_date', 'Company']).mean().reset_index()
label2 = df[['orig_perm_id', 'Company','mine_id', 'grass', 'start_date']].groupby(['orig_perm_id','mine_id','start_date', 'Company']).mean().reset_index()
label3 = df[['orig_perm_id', 'Company','mine_id','flooded_vegetation', 'start_date']].groupby(['orig_perm_id','mine_id', 'start_date', 'Company']).mean().reset_index()
label4 = df[['orig_perm_id', 'Company','mine_id', 'crops', 'start_date']].groupby(['orig_perm_id', 'start_date','mine_id', 'Company']).mean().reset_index().drop_duplicates()
label5 = df[['orig_perm_id', 'Company','mine_id', 'shrub_and_scrub', 'start_date']].groupby(['orig_perm_id', 'start_date', 'mine_id','Company']).mean().reset_index()
label6 = df[['orig_perm_id', 'Company','mine_id', 'built', 'start_date']].groupby(['orig_perm_id', 'start_date','mine_id', 'Company']).mean().reset_index()
label7 = df[['orig_perm_id', 'Company','mine_id', 'bare', 'start_date']].groupby(['orig_perm_id', 'start_date','mine_id', 'Company']).mean().reset_index()
label8 = df[['orig_perm_id', 'Company','mine_id', 'snow_and_ice', 'start_date']].groupby(['orig_perm_id','start_date','mine_id', 'Company']).mean().reset_index()

label0 =label0.drop_duplicates()
label1 =label1.drop_duplicates()
label2 =label2.drop_duplicates()
label3 =label3.drop_duplicates()
label4 =label4.drop_duplicates()
label5 =label5.drop_duplicates()
label6 =label6.drop_duplicates()
label7 =label7.drop_duplicates()
label8 =label8.drop_duplicates()


In [ ]:
group_df = df[['orig_perm_id', 'start_date', 'mine_id', 'Company']].groupby(['orig_perm_id', 'start_date', 'mine_id']).count().reset_index()
group_df = group_df.drop_duplicates()
group_df = group_df.rename(columns={"Company": "Observations"})
group_df

,orig_perm_id,start_date,mine_id,Observations
0,0070008,2017-07-01,00000000000000000611,11
1,0070008,2017-07-01,00000000000000000613,11
2,0070008,2017-07-01,00000000000000000615,11
3,0070008,2017-07-01,00000000000000001e6f,2
4,0070008,2017-07-01,00000000000000003098,4
...,...,...,...,...
9715,Z000781,2023-07-01,0000000000000000016e,8
9716,Z001881,2017-07-01,0000000000000000004c,78
9717,Z001881,2019-07-01,0000000000000000004c,102
9718,Z001881,2021-07-01,0000000000000000004c,64


In [ ]:
df = pd.merge(df, group_df, on=['orig_perm_id', 'start_date', 'mine_id'], how='left').drop_duplicates()

In [ ]:
df = df[['Company', 'start_date', 'Observations', 'orig_perm_id', 'mine_id']].drop_duplicates()
df = df.reset_index()
df = df.drop(['index'], axis=1)

In [ ]:
df = pd.merge(df, label0, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label1, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label2, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label3, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label4, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label5, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label6, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label7, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label8, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()


df.fillna(0, inplace=True)
df

,Company,start_date,Observations,orig_perm_id,mine_id,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,snow_and_ice
0,B & C ENERGY INC,2023-07-01,45,8640180,00000000000000001112,0.031113,0.623011,0.059563,0.032779,0.065345,0.062902,0.035015,0.032943,0.036427
1,HERBERT WELLS,2023-07-01,2,0320080,000000000000000020c7,0.032205,0.539393,0.055222,0.030944,0.088415,0.041989,0.070858,0.030897,0.037347
2,GLOBAL ENERGY GROUP LLC,2023-07-01,73,8260599,000000000000000008a4,0.030102,0.673109,0.048175,0.028032,0.050975,0.045576,0.041335,0.034867,0.034348
3,TRINITY COAL CO INC,2023-07-01,12,0260103,0000000000000000201c,0.032591,0.509787,0.166916,0.035684,0.080755,0.047550,0.033314,0.030661,0.034749
4,HERMITAGE COAL CO INC,2023-07-01,2,643677X,000000000000000003af,0.033055,0.732339,0.035516,0.031776,0.029436,0.037524,0.028389,0.037602,0.034282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9715,LO-MING COAL CORP.,2017-07-01,1,U502397,0000000000000000085a,0.031039,0.587889,0.039089,0.025979,0.048085,0.042070,0.107876,0.037235,0.042496
9716,"APPALACHIAN FUELS, LLC",2017-07-01,67,S304188,0000000000000000057b,0.030835,0.739765,0.032717,0.027254,0.030536,0.037661,0.031239,0.037154,0.032554
9717,"CHICOPEE COAL COMPANY, INC.",2017-07-01,10,S007385,000000000000000007d5,0.027965,0.710045,0.042984,0.025502,0.038912,0.048706,0.032311,0.033129,0.031919
9718,BELVA COAL COMPANY,2017-07-01,10,S009482,00000000000000000869,0.036899,0.544176,0.040389,0.030385,0.046551,0.055853,0.064093,0.120093,0.039626


In [ ]:
df = pd.merge(df, mines,  how='inner', left_on=['orig_perm_id', 'Company'], right_on = ['permit_id', 'company'])

In [ ]:
file_name = 'all_mines'
from google.colab import files
df.to_csv(file_name, encoding = 'utf-8-sig')
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=JcLf2Oeb_StBUjjiwK_kfFJE7e5uOn8oDSM8K0_ZkuM&tc=-OebNKwlijFd5foJMBmkfJ7of7VFifmuQdi-afg3UO0&cc=i3ugGVUZdy4DOE2XX1D3kVmw8thGaIXtHyh3JZ8zhfY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlE-dM1AJefTHYEHAVaLI1LK_5lbylkacaaIIXJdHmsCjnqI9yOBE0

Successfully saved authorization token.


In [ ]:
dataquery = df[['start_date','end_date','year', 'orig_perm_id', 'Company', 'mine_id']].drop_duplicates()
dataquery['end_date'] = dataquery['end_date'].astype(str)
dataquery['start_date'] = dataquery['start_date'].astype(str)

df_lat_long = df[['orig_perm_id', 'start_date','end_date', 'longitude', 'latitude', 'Company', 'mine_id']].groupby(['orig_perm_id', 'end_date', 'Company', 'mine_id']).last()
df_lat_long['start_date'] = df_lat_long['start_date'].astype(str)
dataquery = pd.merge(dataquery, df_lat_long,  how='inner', left_on=['orig_perm_id', 'Company', 'mine_id','start_date'], right_on = ['orig_perm_id', 'Company', 'mine_id','start_date'])